In [1]:
import pandas as pd
import collections
import etl_idesp
pd.set_option('display.max_columns', None)

In [2]:
df_idesp = etl_idesp.etl()
df_idesp.id_escola = df_idesp.id_escola.astype(int)

In [28]:
df_idesp.head(3)

,id_gestao,id_escola,ano_gestao,nota_ano,variacao,anterior_nulo,atual_nulo,unidades_diretoria,nome_diretoria
0,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA
1,esc-10005-ano-2015,10005,2015,3.78,8.620690,0,1,133,ITAPECERICA DA SERRA
2,esc-10005-ano-2016,10005,2016,3.26,-13.756614,0,1,133,ITAPECERICA DA SERRA


In [47]:
LISTA_ANOS = [2014, 2015, 2016, 2017, 2018]
adic_media = {}
adic_media['id_gestao'] = []
adic_media['media_escola'] = []

for nome,dados in df_idesp.groupby('id_escola'):
    nota_total = 0
    contagem = 0
    for ano in LISTA_ANOS:
        if ano == 2014:
            row = dados[dados.ano_gestao == 2014].iloc[0]
            nota_anterior = row.nota_ano * (1 - row.variacao/100)
        else:
            row = dados[dados.ano_gestao == ano-1].iloc[0]
            nota_anterior = row.nota_ano
        row = dados[dados.ano_gestao == ano].iloc[0]
        nota_total += nota_anterior
        contagem += 1
        adic_media['id_gestao'].append(row.id_gestao)
        adic_media['media_escola'].append(nota_total / contagem)
adic_media = pd.DataFrame(data=adic_media)

In [48]:
adic_media.head(3)

,id_gestao,media_escola
0,esc-12-ano-2014,2.549053
1,esc-12-ano-2015,2.839527
2,esc-12-ano-2016,2.953018


# DEPENDENCIAS

In [4]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\instalacao_fisicas\\06_Escolas_Dependencias.csv'
dep = pd.read_csv(arquivo, sep=';')
dep = dep.rename({'CODESC': 'id_escola'}, axis=1)
dep.id_escola = dep.id_escola.astype(int)

In [5]:
len(set(dep.id_escola))

5608

In [6]:
outras_salas = dep.columns[10:]
depend = {}
depend['id_escola'] = []
depend['salas_aula'] = []
depend['outras_salas'] = []
for _, row in dep.iterrows():
    depend['id_escola'].append(row.id_escola)
    depend['salas_aula'].append(int(row.SALAS_AULA))
    soma_outras = 0
    for col in outras_salas:
        soma_outras += int(row.loc[col])
    depend['outras_salas'].append(soma_outras)
depend = pd.DataFrame(depend)

In [7]:
depend.head(3)

,id_escola,salas_aula,outras_salas
0,985429,5,29
1,31045,9,40
2,31112,12,64


# ENDERECO

In [8]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\escolas_enderecos\\11_escolas_enderecos_0.csv'
with open(arquivo, encoding="utf8") as file_in:
    lines = []
    for line in file_in:
        lines.append(line)
dados_lines = []
len_lines = len(lines[0].split(','))
for line in lines[1:]:
    line = line.rstrip().split(',')
    if len(line) == len_lines:  # linhas com numero incorreto de colunas foram descartadas (1)
        dados_lines.append(line)
end = pd.DataFrame(data=dados_lines)
end.columns = lines[0].split(',')
end.head(3)

,nomedep,depadm,mun,codmun,de,CD_ESCOLA,CD_DIRETORIA,CD_DIRETORIA_ESTADUAL,CD_DIRETORIA_SUPVS_PROPR,NM_COMPLETO_ESCOLA,CD_UNIDADE,DS_ENDERECO,COMPLEMENTO,NUMERO,CD_ORIGEM_UNIDADE,CD_ORIGEM_ESCOLA,CD_ORIGEM_ENDERECO,LATITUDE,LONGITUDE
0,ESTADUAL - SE,1,SAO PAULO,100,NORTE 1,36444,10101,10101,10101,GENESIO DE ALMEIDA MOURA DOUTOR,37806,DOMINGOS AREVALO,RUA,862,37806,36444,37866,-23.447,-46.6967
1,ESTADUAL - SE,1,SAO PAULO,100,LESTE 5,1582,10205,10205,10205,DOMINGOS FAUSTINO SARMIENTO,24902,21 DE ABRIL,RUA,970,24902,1582,24962,-23.5422317000605,-46.6057931815944
2,ESTADUAL - SE,1,SAO PAULO,100,LESTE 5,2173,10205,10205,10205,JOAO BORGES PROFESSOR,24901,ITAPURA,RUA,976,24901,2173,24961,-23.5473,-46.566


In [9]:
end = end.rename({'CD_ESCOLA': 'id_escola'}, axis=1)
end = end.rename({'LONGITUDE\n': 'LONGITUDE'}, axis=1)
end.id_escola = end.id_escola.astype(int)
end = end.drop_duplicates(subset=['id_escola'])

In [10]:
dic_mun = collections.Counter(end.mun)
end['unidades_municipio'] = end.mun.apply(lambda x: dic_mun[x])

In [11]:
end = end.loc[:,['id_escola','LATITUDE', 'LONGITUDE','unidades_municipio','mun']]

In [12]:
end.head(3)

,id_escola,LATITUDE,LONGITUDE,unidades_municipio,mun
0,36444,-23.447,-46.6967,6626,SAO PAULO
1,1582,-23.5422317000605,-46.6057931815944,6626,SAO PAULO
2,2173,-23.5473,-46.566,6626,SAO PAULO


# CLUSTER

In [13]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\cluster_de_escolas_similares\\12_MMR_Clusters.csv'
cluster = pd.read_csv(arquivo, sep=';')
cluster.CD_ESCOLA = cluster.CD_ESCOLA.astype(int)
cluster = cluster.rename({'CD_ESCOLA': 'id_escola'}, axis=1)
cluster = cluster.drop_duplicates(subset=['id_escola'])

In [14]:
len(set(cluster.id_escola))

5056

In [15]:
cluster = cluster.loc[:,['id_escola','NR_CLUSTER']]

In [16]:
cluster.head(3)

,id_escola,NR_CLUSTER
0,24454,1
1,24466,1
2,24491,3


# CLASSES

In [17]:
arquivo = 'dados-VERSAO_1\matriculas\/10_Escolas_Classes_Qtde_Alunos.csv'
classes = pd.read_csv(arquivo, sep=';')

In [18]:
classes = classes.rename({'COD_ESC': 'id_escola'}, axis=1)
classes.ANO = classes.ANO.astype(int)
classes = classes[classes.ANO == 2019]

In [19]:
clas = {}
clas['id_escola'] = []
clas['qtd_alunos'] = []
clas['tipos_classes'] = []

for nome, dados in classes.groupby('id_escola'):
    clas['id_escola'].append(nome)
    clas['qtd_alunos'].append(dados.QTDE_ALUNOS.sum())
    clas['tipos_classes'].append(len(set(dados.TIPOCLASSE)))
clas = pd.DataFrame(data=clas)

In [20]:
clas.head(3)

,id_escola,qtd_alunos,tipos_classes
0,12,560,1
1,24,2167,1
2,36,2274,2


# SCORE DOS GESTORES
- Crescente com base no ano atual.